In [1]:
# packages

import numpy as np
import pandas as pd

In [26]:
FEE_HAIRCUT = 0.997
EXCHANGES = [
    {"input_token": 1, "input_liquidity": 2.1, "output_token": 2, "output_liquidity": 1.2, "haircut": FEE_HAIRCUT},
    {"input_token": 2, "input_liquidity": 2.2, "output_token": 3, "output_liquidity": 1.3, "haircut": FEE_HAIRCUT},
    {"input_token": 3, "input_liquidity": 2.3, "output_token": 4, "output_liquidity": 1.4, "haircut": FEE_HAIRCUT},
    {"input_token": 4, "input_liquidity": 2.4, "output_token": 1, "output_liquidity": 1.1, "haircut": FEE_HAIRCUT}, ]


In [27]:
SWAP_CURVE_X = np.exp(np.linspace(0, 1, 10)) - 1
#print(SWAP_CURVE_X)

In [28]:
# Define forward and backward swap calculators
from typing import Union

def forward_swap(x:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(x, float):
        assert x >= 0
    elif isinstance(x, np.ndarray):
        assert np.all(x) >= 0
    assert haircut > 0
    return haircut * x /(1 + haircut * x)

def backward_swap(y:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(y, float):
        assert y >= 0.0
        assert y < 1.0
    elif isinstance(y, np.ndarray):
        assert np.all(y) >= 0
        assert np.all(y < 1.0)
    assert haircut > 0
    return y /(haircut - haircut * y)



In [29]:
def generate_ITL(exchange_list:list[dict]=EXCHANGES) -> dict:
    ITL = {}
    for i in range(len(exchange_list)-1):
        assert exchange_list[i]['output_token'] == exchange_list[i+1]['input_token']
        ITL[exchange_list[i+1]['input_token']] = exchange_list[i]['output_liquidity']/ exchange_list[i+1]['input_liquidity']
    return ITL



In [30]:
ITL = generate_ITL()
print(ITL[2])
exchange_list = EXCHANGES

0.5454545454545454


In [31]:
# Define swap curves
x_curve = SWAP_CURVE_X
y_curve = forward_swap(x_curve)
test_x_curve = backward_swap(y_curve)

# Check if forward_swap equals inverse of backward_swap
inverse_test = np.allclose(x_curve, test_x_curve)
print(f"Swap curves defined. \nForward and Backward are inverses: {inverse_test}")

Swap curves defined. 
Forward and Backward are inverses: True


In [37]:
# Recursive compute

# Exchange 1 -> 2
# token_flow = exchange_list[0]['output_token']
token_in = exchange_list[0]['input_token']
token_in_liquidity = exchange_list[0]['input_liquidity']
print(f"token_in: {token_in} token_in_liquidity: {token_in_liquidity}")

# print(token_flow)
# print(token_out)
# assert token_flow == token_out
# intermediate_token_liquidity = ITL[token_flow]
# print(f"Token liquidity for token-{token_flow} = {intermediate_token_liquidity}")

delta_1 = x_curve                                               # delta_1 can be any amount
X_1 = delta_1  / token_in_liquidity                             # Forward swap argument: delta_1 / a_{1,2})

xi_indx = np.searchsorted(x_curve, X_1, side='right')           # x_curve[xi_indx] > X_1
eta_1 = y_curve[xi_indx]
for i in range(len(x_curve)):
    if xi_indx[i] == len(x_curve):
        xi_indx -= 1
    # print(f"{i} xi: {x_curve[xi_indx[i]]:6.4f},  X_1: {X_1[i]:6.4f}, xi_indx: {xi_indx[i]} ")
    print(f"{i} delta_1: {delta_1[i]:6.4f},  eta_1: {eta_1[i]:6.4f}")



token_in: 1 token_in_liquidity: 2.1
0 delta_1: 0.0000,  eta_1: 0.1049
1 delta_1: 0.1175,  eta_1: 0.1049
2 delta_1: 0.2488,  eta_1: 0.1988
3 delta_1: 0.3956,  eta_1: 0.1988
4 delta_1: 0.5596,  eta_1: 0.2829
5 delta_1: 0.7429,  eta_1: 0.2829
6 delta_1: 0.9477,  eta_1: 0.3581
7 delta_1: 1.1766,  eta_1: 0.4255
8 delta_1: 1.4324,  eta_1: 0.4255
9 delta_1: 1.7183,  eta_1: 0.4858


In [42]:
# Exchange 2 -> 3
token_flow = exchange_list[0]['output_token']
token_out = exchange_list[1]['input_token']

assert token_flow == token_out
intermediate_token_liquidity = ITL[token_flow]
print(f"Token liquidity for token-{token_flow} = {intermediate_token_liquidity}")

delta_2 = token_flow * eta_1                               # delta_2 upper bound: delta_2 < eta_1 * a_{2,1}
X_2 = eta_1 * intermediate_token_liquidity                 # Discrete swap argument: eta_1 * (a_{2,1}/a_{2,3})

xi_indx = np.searchsorted(x_curve, X_2, side='right')      # x_curve[xi_indx] > X_2
eta_2 = y_curve[xi_indx]
for i in range(len(x_curve)):
    if xi_indx[i] == len(x_curve):
        xi_indx -= 1
    print(f"{i} xi: {x_curve[xi_indx[i]]:6.4f},  X_2: {X_2[i]:6.4f}, xi_indx: {xi_indx[i]} ")
    #print(f"{i} delta_2: {delta_2[i]:6.4f},  eta_2: {eta_2[i]:6.4f}")


Token liquidity for token-2 = 0.5454545454545454
0 xi: 0.1175,  X_2: 0.0572, xi_indx: 1 
1 xi: 0.1175,  X_2: 0.0572, xi_indx: 1 
2 xi: 0.1175,  X_2: 0.1084, xi_indx: 1 
3 xi: 0.1175,  X_2: 0.1084, xi_indx: 1 
4 xi: 0.2488,  X_2: 0.1543, xi_indx: 2 
5 xi: 0.2488,  X_2: 0.1543, xi_indx: 2 
6 xi: 0.2488,  X_2: 0.1953, xi_indx: 2 
7 xi: 0.2488,  X_2: 0.2321, xi_indx: 2 
8 xi: 0.2488,  X_2: 0.2321, xi_indx: 2 
9 xi: 0.3956,  X_2: 0.2650, xi_indx: 3 


In [43]:
# Exchange 3 -> 4
token_flow = exchange_list[1]['output_token']
token_out = exchange_list[2]['input_token']

assert token_flow == token_out
intermediate_token_liquidity = ITL[token_flow]
print(f"Token liquidity for token-{token_flow} = {intermediate_token_liquidity}")

delta_3 = token_flow * eta_2                               # delta_3 upper bound: delta_3 < eta_2 * a_{3,2}
X_3 = eta_2 * intermediate_token_liquidity                 # Discrete swap argument: eta_2 * (a_{3,2}/a_{3,4})

xi_indx = np.searchsorted(x_curve, X_3, side='right')      # x_curve[xi_indx] > X_2
eta_3 = y_curve[xi_indx]
for i in range(len(x_curve)):
    if xi_indx[i] == len(x_curve):
        xi_indx -= 1
    print(f"{i} xi: {x_curve[xi_indx[i]]:6.4f},  X_3: {X_3[i]:6.4f}, xi_indx: {xi_indx[i]} ")
    #print(f"{i} delta_2: {delta_2[i]:6.4f},  eta_2: {eta_2[i]:6.4f}")


Token liquidity for token-3 = 0.5652173913043479
0 xi: 0.1175,  X_3: 0.0593, xi_indx: 1 
1 xi: 0.1175,  X_3: 0.0593, xi_indx: 1 
2 xi: 0.1175,  X_3: 0.0593, xi_indx: 1 
3 xi: 0.1175,  X_3: 0.0593, xi_indx: 1 
4 xi: 0.1175,  X_3: 0.1124, xi_indx: 1 
5 xi: 0.1175,  X_3: 0.1124, xi_indx: 1 
6 xi: 0.1175,  X_3: 0.1124, xi_indx: 1 
7 xi: 0.1175,  X_3: 0.1124, xi_indx: 1 
8 xi: 0.1175,  X_3: 0.1124, xi_indx: 1 
9 xi: 0.2488,  X_3: 0.1599, xi_indx: 2 


In [44]:
# Exchange 4 -> 1
token_flow = exchange_list[2]['output_token']
token_out = exchange_list[3]['input_token']

assert token_flow == token_out
intermediate_token_liquidity = ITL[token_flow]
print(f"Token liquidity for token-{token_flow} = {intermediate_token_liquidity}")

delta_4 = token_flow * eta_3                               # delta_4 upper bound: delta_4 < eta_3 * a_{4,3}
X_4 = eta_3 * intermediate_token_liquidity                 # Discrete swap argument: eta_3 * (a_{4,3/a_{4,1})

xi_indx = np.searchsorted(x_curve, X_4, side='right')      # x_curve[xi_indx] > X_2
eta_3 = y_curve[xi_indx]
for i in range(len(x_curve)):
    if xi_indx[i] == len(x_curve):
        xi_indx -= 1
    print(f"{i} xi: {x_curve[xi_indx[i]]:6.4f},  X_3: {X_4[i]:6.4f}, xi_indx: {xi_indx[i]} ")
    #print(f"{i} delta_2: {delta_2[i]:6.4f},  eta_2: {eta_2[i]:6.4f}")


Token liquidity for token-4 = 0.5833333333333334
0 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
1 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
2 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
3 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
4 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
5 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
6 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
7 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
8 xi: 0.1175,  X_3: 0.0612, xi_indx: 1 
9 xi: 0.1175,  X_3: 0.1160, xi_indx: 1 
